This notebook will be mainly used for the capstone project to compare various neighborhoods in Toronto.

In [9]:
import pandas as pd
import numpy as np
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


1. Extraction of data from Wikipedia to a clean DataFrame

In [17]:
#extract data from Wikipedia
import requests
wikipedia_link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
raw_wikipedia_page = requests.get(wikipedia_link).text

In [12]:
# import the beautifulsoup library
from bs4 import BeautifulSoup
soup = BeautifulSoup(raw_wikipedia_page,'lxml')
toronto_data = BeautifulSoup(raw_wikipedia_page, 'lxml')

In [14]:
# create a new DataFrame
column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)

In [15]:
# loop through to find postcode, borough and neighborhood 
content = Toronto_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

In [16]:
# clean the dataframe 
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1
                                 
df_toronto = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_toronto.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


2. Latitude and the longitude coordinates of each neighborhood. 


In [18]:
import csv
import json
import xml

In [64]:
# Get the latitude and longitude coordinates
filename = "http://cocl.us/Geospatial_data"
headers = ["PostalCode","Latitude","Longitude"]

df_ll = pd.read_csv(filename, names = headers)
df_ll.head()

,PostalCode,Latitude,Longitude
0,Postal Code,Latitude,Longitude
1,M1B,43.8066863,-79.1943534
2,M1C,43.7845351,-79.1604971
3,M1E,43.7635726,-79.1887115
4,M1G,43.7709921,-79.2169174


In [ ]:
# Concat the two dataframes

In [69]:
df_final =pd.concat([df_toronto, df_ll], axis=1)
df_final.head()

,Postalcode,Borough,Neighborhood,PostalCode,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",Postal Code,Latitude,Longitude
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1B,43.8066863,-79.1943534
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1C,43.7845351,-79.1604971
3,M1G,Scarborough,Woburn,M1E,43.7635726,-79.1887115
4,M1H,Scarborough,Cedarbrae,M1G,43.7709921,-79.2169174


In [70]:
del df_final['PostalCode']
df_final.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",Latitude,Longitude
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.8066863,-79.1943534
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7845351,-79.1604971
3,M1G,Scarborough,Woburn,43.7635726,-79.1887115
4,M1H,Scarborough,Cedarbrae,43.7709921,-79.2169174
